## Libraries

In [1]:
import os
import datetime
import regex as re
import glob
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import shutil
from pathlib import Path
from datetime import datetime
import pandas as pd
import html
import numpy as np
import nltk
nltk.download('stopwords')
from string import punctuation
stopwords = set(nltk.corpus.stopwords.words('english'))## Download Data to Dataframe

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/halledavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Download Data to Dataframe

In [6]:
#os.chdir('C:\\Users\\halle.davis\\Downloads\\reports') ## Halle's work folder path
os.chdir('/Users/halledavis/Desktop/reports') ## Halle's personal folder path
#os.chdir('') ## Claire's folder path
#os.chdir('') ## Summer's folder path

In [7]:
file_list = glob.glob(os.path.join(os.getcwd(), "*.txt"))
start_time = datetime.now()

reports = []

for file_path in file_list:
    with open(file_path) as f_input:
        text = (f_input.read())
        file_name = file_path.split("reports")[-1]
        reports.append(
            {
                'filename': file_name,
                'alltext': text
            }
        )
        
data = pd.DataFrame(reports)

end_time = datetime.now()
print("Started At:", start_time, "\nEnded At:", end_time)

Started At: 2023-06-18 16:36:26.680794 
Ended At: 2023-06-18 16:36:26.821621


## Preprocessing

### Get Title

In [8]:
data['title'] = data.alltext.astype(str).str.extract(r': (.*)')

In [9]:
data.head(20)

,filename,alltext,title
0,/FATALS2004FTL04m22-23.txt,\n\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmeta...,"Fatality #22 & 23 - December 05, 2004 - Falli..."
1,/FATALS2006FTL06m25.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #25 - December 15, 2006"
2,/FATALS2005FTL05m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #7 - April 1, 2005"
3,/FATALS2004FTL04m15.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #15 - August 18, 2004"
4,/FATALS2004FTL04m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #1 - January 29, 2004 -"
5,/FATALS2007FTL07m23.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #23 - September 7, 2007 - Machinery ..."
6,/FATALS2005FTL05m13.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #13 - May 31, 2005 - Powered Haulage ..."
7,/FATALS2003FTL03m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #10 - May 21, 2003 -"
8,/FATALS2000FTL00M32.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #32 - August 09, 2000 - Powered Haul..."
9,/FATALS2002FTL02m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #02 - January 12, 2002 - Powered Ha..."


### Get Cause

#### Causes include
ELECTRICAL - Accidents in which electric current is most directly responsible for the resulting accident.

ENTRAPMENT - In accidents involving no injuries or nonfatal injuries which are not serious, entrapment of mine workers takes precedence over roof falls, explosives accidents, inundations, etc. If a roof fall results in an entrapment accident, the accident classification is ï¿½Entrapment.

EXPLODING VESSELS UNDER PRESSURE - These are accidents caused by explosion of air hoses, air tanks, hydraulic lines, hydraulic hoses, and other accidents precipitated by exploding vessels.

EXPLOSIVES AND BREAKING AGENTS - Accidents involving the detonation of manufactured explosives, Airdox, or Cardox, that can cause flying debris, concussive forces, or fumes.

FALLING, ROLLING, OR SLIDING ROCK OR MATERIAL OF ANY KIND -Injuries caused directly by falling material require great care in classification. Remember that it is the accident we want to classify. If material was set in motion by machinery, haulage equipment, or hand tools, or while material is being handled or disturbed, etc., charge the force that set the material in motion. For example, where a rock was pushed over a highwall by a dozer and the rock hit another rock which struck and injured a worker - charge the accident to the dozer. Charge the accident to that which most directly caused the resulting accident. Without the dozer, there would have been no resulting accident. This includes accidents caused by improper blocking of equipment under repair or inspection.

FALL OF FACE, RIB, SIDE OR HIGHWALL - Accidents in this classification include falls of material (from in-place) while barring down or placing props; also pressure bumps and bursts. Since pressure bumps and bursts which cause accidents are infrequent, they are not given a separate category. Not included are accidents in which the motion of machinery or haulage equipment caused the fall either directly or by knocking out support; such accidents are classified as machinery or haulage, whichever is appropriate.

FALL OF ROOF OR BACK - Underground accidents which include falls while barring down or placing props; also pressure bumps and bursts. Not included are accidents in which the motion of machinery or haulage equipment caused the fall either directly or by knocking out support; such falls are classified as machinery or haulage, whichever is appropriate.

FIRE - In underground mines, an unplanned fire not extinguished within 10 minutes of discovery; in surface mines and surface areas of underground mines, an unplanned fire not extinguished within 30 minutes of discovery;

HANDLING MATERIAL - (Lifting, pulling, pushing, shoveling material.) The material may be in bags or boxes, or loose sand, coal, rock, timber, etc. The accident must have been most directly caused by handling material.

HAND TOOLS - Accidents related to non-powered tools when being used as hand tools. Do not include electric tools or air-powered tools.

HOISTING - Damage to hoisting equipment in a shaft or slope which endangers an individual or interferes with use of the equipment for more than 30 minutes. Hoisting may also be the classification where a victim was injured by hoisting equipment but there was no damage to the equipment. Accidents involving cages, skips, buckets, or elevators. The accident results from the action, motion, or failure of the hoisting equipment or mechanism. Included is equipment such as derricks and cranes only when used in shaft sinking; suspended work platforms in shafts; mine cars being lowered or raised by hoisting equipment on slopes or inclines; a skip squeezed between timbers resulting in an accident; or an ore bucket tipped for any reason causing an accident.

IGNITION OR EXPLOSION OF GAS OR DUST - Accidents resulting as a consequence of the ignition or explosion of gas or dust. Included are exploding gasoline vapors, space heaters, or furnaces.
Methane Ignition - A methane ignition occurs when methane burns without producing destructive forces. Damage resulting from an ignition is limited to that caused by flame and heat. Personnel in the immediate vicinity of an ignition may be burned and line brattice or other materials in close proximity may be discolored, melted or burned. Ignitions generally involve small quantities of methane and are usually confined to a small area; however, in the case of methane roof layering, flame spread may be more extensive.
Methane Explosion - A methane explosion occurs when methane is ignited and burns violently. The flame of the explosion accelerates rapidly, heating the environment and causing destructive forces. Evidence of the destructive forces may be manifest on victims, equipment, structures, etc. Witnesses to an explosion may hear the noise generated by the resulting sound pressure wave.

IMPOUNDMENT - An unstable condition at an impoundment, refuse pile, or culm bank which requires emergency action in order to prevent failure, or which causes individuals to evacuate an area. Also the failure of an impoundment, refuse pile, or culm bank.

INUNDATION - An unplanned inundation of a mine by a liquid or gas. The mine may be either a surface or underground operation.

MACHINERY - Accidents that result from the action or motion of machinery or from failure of component parts. Included are all electric and air-powered tools and mining machinery such as drills, tuggers, slushers, draglines, power shovels, loading machines, compressors, etc. Include derricks and cranes except when they are used in shaft sinking (see HOISTING) or mobile cranes traveling with a load (see POWERED HAULAGE).

NON-POWERED HAULAGE - Accidents related to motion of non-powered haulage equipment. Included are accidents involving wheelbarrows, manually pushed mine cars and trucks, etc.

POWERED HAULAGE - Haulage includes motors and rail cars, conveyors, belt feeders, longwall conveyors, bucket elevators, vertical manlifts, self-loading scrapers or pans, shuttle cars, haulage trucks, front-end loaders, load-haul- dumps, forklifts, cherry pickers, mobile cranes if traveling with a load, etc. The accident is caused by the motion of the haulage unit. Include accidents that are caused by an energized or moving unit or failure of component parts. If a car dropper suffers an injury as a result of falling from a moving car, charge the accident to haulage.

SLIP OR FALL OF PERSON - Includes slips or falls from an elevated position or at the same level while getting on or off machinery or haulage equipment that is not moving. Also includes slips or falls while servicing or repairing equipment or machinery. Includes stepping in a hole.

STEPPING OR KNEELING ON OBJECT - Accidents are classified in this category only where the object stepped or kneeled on contributed most directly to the accident.

STRIKING OR BUMPING - This classification is restricted to those accidents in which an individual, while moving about, strikes or bumps an object but is not handling material, using hand tools, or operating equipment.

OTHER - Accidents not elsewhere classified. This is a last resort category.

In [10]:
#Cause Keywords

electric = ['electric', 'electrical']
entrapment = ['entrapment']
explodingvessels = ['vessels', 'pressure']
explosives = ['explosives', 'agents']
fallingmaterial = ['material', 'rock']
fallofface = ['face', 'rib', 'side', 'highwall']
fallofroof = ['roof', 'back']
fire = ['fire']
handlingmaterial = ['handling']
handtools = ['hand', 'handtools']
hoisting = ['hoist', 'hositing']
ignition = ['ignition', 'gas', 'dust', 'ignition/explosion', 'methane']
impoundment = ['impoundment']
inundation = ['inundation']
machinery = ['machinery']
nonpoweredhaulage = ['non-powered']
poweredhaulage = ['powered']
slipfall = ['slip', 'fall', 'slip/fall', '/fall']
stepping = ['stepping', 'kneeling']
striking = ['striking', 'bumping']
other = ['other']

In [11]:
def causeclassifier(sentence):
    if any(word in electric for word in sentence.lower().split()):
        return 'Electric'
    elif any(word in entrapment for word in sentence.lower().split()):
        return 'Entrapment'
    elif any(word in explodingvessels for word in sentence.lower().split()):
        return 'Exploding Vessels'
    elif any(word in explosives for word in sentence.lower().split()):
        return 'Explosives'
    elif any(word in fallingmaterial for word in sentence.lower().split()):
        return 'Falling Material'
    elif any(word in fallofface for word in sentence.lower().split()):
        return 'Fall of Face'
    elif any(word in fallofroof for word in sentence.lower().split()):
        return 'Fall of Roof'
    elif any(word in fire for word in sentence.lower().split()):
        return 'Fire'
    elif any(word in handlingmaterial for word in sentence.lower().split()):
        return 'Handling Material'
    elif any(word in handtools for word in sentence.lower().split()):
        return 'Hand Tools'
    elif any(word in hoisting for word in sentence.lower().split()):
        return 'Hoisting'
    elif any(word in ignition for word in sentence.lower().split()):
        return 'Ignition/Explosion'
    elif any(word in inundation for word in sentence.lower().split()):
        return 'Inundation'
    elif any(word in machinery for word in sentence.lower().split()):
        return 'Machinery'
    elif any(word in nonpoweredhaulage for word in sentence.lower().split()):
        return 'Non-Powered Haulage'
    elif any(word in poweredhaulage for word in sentence.lower().split()):
        return 'Powered Haulage'
    elif any(word in slipfall for word in sentence.lower().split()):
        return 'Slip/Fall'
    elif any(word in stepping for word in sentence.lower().split()):
        return 'Stepping'
    elif any(word in striking for word in sentence.lower().split()):
        return 'Striking'
    elif any(word in other for word in sentence.lower().split()):
        return 'Other'
    else:
        return 'Other'

In [12]:
data['cause'] = data['title'].astype(str).apply(causeclassifier)    

In [13]:
data.head(20)

,filename,alltext,title,cause
0,/FATALS2004FTL04m22-23.txt,\n\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmeta...,"Fatality #22 & 23 - December 05, 2004 - Falli...",Falling Material
1,/FATALS2006FTL06m25.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #25 - December 15, 2006",Other
2,/FATALS2005FTL05m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #7 - April 1, 2005",Other
3,/FATALS2004FTL04m15.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #15 - August 18, 2004",Other
4,/FATALS2004FTL04m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #1 - January 29, 2004 -",Other
5,/FATALS2007FTL07m23.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #23 - September 7, 2007 - Machinery ...",Machinery
6,/FATALS2005FTL05m13.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #13 - May 31, 2005 - Powered Haulage ...",Powered Haulage
7,/FATALS2003FTL03m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #10 - May 21, 2003 -",Other
8,/FATALS2000FTL00M32.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #32 - August 09, 2000 - Powered Haul...",Powered Haulage
9,/FATALS2002FTL02m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #02 - January 12, 2002 - Powered Ha...",Powered Haulage


### Get Year

In [14]:
data['year']  = data.alltext.astype(str).str.extract(r'(200\d)')

In [15]:
data.tail(20)

,filename,alltext,title,cause,year
212,/FATALS2003FTL03m20.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #20 - October 1, 2003",Other,2003
213,/FATALS2002FTL02m32.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #32 - October 17, 2002 - Machinery -...",Machinery,2002
214,/FATALS2001FTL01m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #10 - Slip/Fall of Person - AL - Lim...,Slip/Fall,2001
215,/FATALS2003FTL03m21.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #21 - April 21, 2003 - Powered Haula...",Powered Haulage,2003
216,/FATALS2001FTL01m11.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #11 - Slip/Fall of Person - GA - Gra...,Falling Material,2001
217,/FATALS2002FTL02m33.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #33 - October 21, 2002 - Powered Hau...",Powered Haulage,2002
218,/FATALS2002FTL02m27.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #27 - September 16, 2002 - Electrical...",Electric,2002
219,/FATALS2001FTL01m05.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #5 - 03/20/2001 - Ignition/Explosion of ...,Ignition/Explosion,2001
220,/FATALS2000FTL00M17.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #17 - Machinery - Amethyst - NV - Jo...,Machinery,2000
221,/FATALS2003FTL03m09.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #9 - May 15, 2003 -",Other,2003


### Clean & Tokenize Text

In [16]:
def tokenize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*', text)

In [17]:
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

In [18]:
def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]

In [19]:
pipeline = [str.lower, tokenize, remove_stop]
def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

In [20]:
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    # numbers
    text = re.sub(r'\d+', '', text)
    return text.strip()

In [21]:
data['tokens'] = data['alltext'].apply(prepare, pipeline=pipeline)

In [22]:
data["cleantext"] = data["alltext"].apply(clean)
data["cleantext"] = data["cleantext"].apply(str.lower)
data["cleantext"] = data["cleantext"].apply(remove_punctuation)

In [23]:
data.head(20)

,filename,alltext,title,cause,year,tokens,cleantext
0,/FATALS2004FTL04m22-23.txt,\n\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmeta...,"Fatality #22 & 23 - December 05, 2004 - Falli...",Falling Material,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
1,/FATALS2006FTL06m25.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #25 - December 15, 2006",Other,2006,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
2,/FATALS2005FTL05m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #7 - April 1, 2005",Other,2005,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
3,/FATALS2004FTL04m15.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #15 - August 18, 2004",Other,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
4,/FATALS2004FTL04m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #1 - January 29, 2004 -",Other,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
5,/FATALS2007FTL07m23.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #23 - September 7, 2007 - Machinery ...",Machinery,2007,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
6,/FATALS2005FTL05m13.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #13 - May 31, 2005 - Powered Haulage ...",Powered Haulage,2005,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...
7,/FATALS2003FTL03m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #10 - May 21, 2003 -",Other,2003,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
8,/FATALS2000FTL00M32.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #32 - August 09, 2000 - Powered Haul...",Powered Haulage,2000,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
9,/FATALS2002FTL02m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #02 - January 12, 2002 - Powered Ha...",Powered Haulage,2002,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...


### Get Text by Category

In [24]:
# Get Overview

data['overview']  = data.cleantext.astype(str).str.extract(r'overview(.*) general info')

In [25]:
# Get General Information

data['geninfo']  = data.cleantext.astype(str).str.extract(r'information(.*)description of') #sometimes its "the accident" and sometimes its just accident

In [26]:
# Description of Accident

data['description']  = data.cleantext.astype(str).str.extract(r'description of(.*)investigation of') #sometimes its "the accident" and sometimes its just accident

In [27]:
# Other Info
# contains information from the investigation, root cause, and discussion segments
# did not separate because headers vary in order and presence, so was difficult to parse out those exact sections

data['otherinfo']  = data.cleantext.astype(str).str.extract(r'investigation of(.*)conclusion') #sometimes its "the accident" and sometimes its just accident



In [28]:
# Failed Attempt to Pull Out Investigation Only -- doesn't take into consideration ordering differences
# investigation = []

#for value in data['cleantext']:
#    if 'root cause analysis' in value:
#        investigation.append(re.findall(r'investigation of(.*)root cause analysis', value))
#    else:
#        investigation.append(re.findall(r'investigation of(.*) discussion', value))

# data['investigation'] = investigation

In [29]:
data['conclusion']  = data.cleantext.astype(str).str.extract(r'conclusion(.*)enforcement')

In [30]:
data['enforcement']  = data.cleantext.astype(str).str.extract(r'enforcement action(.*)appendix')

In [31]:
data['overview'] = data['overview'].astype(str)
data['overview_tokens'] = data['overview'].apply(prepare, pipeline=pipeline)

data['geninfo'] = data['geninfo'].astype(str)
data['geninfo_tokens'] = data['geninfo'].apply(prepare, pipeline=pipeline)

data['otherinfo'] = data['otherinfo'].astype(str)
data['otherinfo_tokens'] = data['otherinfo'].apply(prepare, pipeline=pipeline)

data['conclusion'] = data['conclusion'].astype(str)
data['conclusion_tokens'] = data['conclusion'].apply(prepare, pipeline=pipeline)

data['enforcement'] = data['enforcement'].astype(str)
data['enforcement_tokens'] = data['enforcement'].apply(prepare, pipeline=pipeline)

In [32]:
data.head(20)

,filename,alltext,title,cause,year,tokens,cleantext,overview,geninfo,description,otherinfo,conclusion,enforcement,overview_tokens,geninfo_tokens,otherinfo_tokens,conclusion_tokens,enforcement_tokens
0,/FATALS2004FTL04m22-23.txt,\n\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmeta...,"Fatality #22 & 23 - December 05, 2004 - Falli...",Falling Material,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...,united states department of labor mine safety...,robinson operation a surface copper ore opera...,the accident on the day of the accident micha...,the accident msha was notified of the acciden...,the accident occurred because safe work proce...,order no was issued on december under prov...,"[united, states, department, labor, mine, safe...","[robinson, operation, surface, copper, ore, op...","[accident, msha, notified, accident, december,...","[accident, occurred, safe, work, procedures, e...","[order, issued, december, provisions, section,..."
1,/FATALS2006FTL06m25.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #25 - December 15, 2006",Other,2006,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...,united states department of labor mine safety...,red dog mine a surface lead and zinc mine ope...,accident on the day of the accident jeffrey h...,accident msha was notified of the accident at...,the accident occurred because the muckpile fa...,order no was issued on december under the ...,"[united, states, department, labor, mine, safe...","[red, dog, mine, surface, lead, zinc, mine, op...","[accident, msha, notified, accident, december,...","[accident, occurred, muckpile, face, trimmed, ...","[order, issued, december, provisions, section,..."
2,/FATALS2005FTL05m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #7 - April 1, 2005",Other,2005,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...,united states department of labor mine safety...,kimballton plant # an underground limestone o...,accident on the day of the accident robert bu...,the accident msha was notified of the acciden...,the accident was caused by the failure to eva...,chemical lime company of virginia inc order n...,"[united, states, department, labor, mine, safe...","[kimballton, plant, underground, limestone, op...","[accident, msha, notified, accident, pm, april...","[accident, caused, failure, evaluate, work, pr...","[chemical, lime, company, virginia, inc, order..."
3,/FATALS2004FTL04m15.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #15 - August 18, 2004",Other,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...,united states department of labor mine safety...,clarksville plant a cement plant and quarry w...,accident on the day of the accident jerry huc...,accident msha was notified of the accident at...,the accident occurred because safe work proce...,s order no was issued on august under the p...,"[united, states, department, labor, mine, safe...","[clarksville, plant, cement, plant, quarry, lo...","[accident, msha, notified, accident, pm, augus...","[accident, occurred, safe, work, procedures, e...","[order, issued, august, provisions, section, k..."
4,/FATALS2004FTL04m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #1 - January 29, 2004 -",Other,2004,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metalnonmetal mine fatal accident investi...,united states department of labor mine safety...,holcim us inc holly hill facility a surface q...,accident on the day of the accident kenneth l...,the accident msha was notified of the acciden...,nan,nan,"[united, states, department, labor, mine, safe...","[holcim, us, inc, holly, hill, facility, surfa...","[accident, msha, notified, accident, pm, janua...",[nan],[nan]
5,/FATALS2007FTL07m23.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality

## EDA

In [33]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :    
    counter = Counter()
    tokens.map(counter.update)
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    
    counter_df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    
    num_tokens = sum(freq_df['freq'])
    num_unique_tokens = freq_df.shape[0]
    lexical_diversity = num_unique_tokens / num_tokens
    num_characters = sum((counter_df['index'].str.len()) * counter_df[0])
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(f"The top 5 most common words are")
        print(counter.most_common(5))
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])

In [34]:
descriptive_stats(data["tokens"])

There are 318004 tokens in the data.
There are 13549 unique tokens in the data.
There are 2062967 characters in the data.
The lexical diversity is 0.043 in the data.
The top 5 most common words are
[('mine', 6642), ('accident', 4898), ('safety', 4332), ('health', 3094), ('msha', 2036)]


[318004, 13549, 0.04260638230965648, 2062967]